In [1]:
import json
import minsearch
from datetime import datetime
import ollama
import google.generativeai as genai
import os

In [2]:
with open('./documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)
    
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [3]:
def build_prompt(query, search_result):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, output NONE
    
    QUESTION: {question}
    CONTEXT:
    {context}
    """.strip()
    context = ''
    for doc in search_result:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    prompt = prompt_template.format(question=query, context=context)
    return prompt.strip()

def llm(prompt):
    model_name = 'llama3.2:1b'
    print(model_name)
    response = ollama.chat(model=model_name, messages=[{'role' : 'user', \
    'content' : prompt}])
    return response['message']['content'] # ollama version

def rag(query):
    search_result = search(query)
    prompt = build_prompt(query, search_result)
    answer = llm(prompt)
    return answer


In [4]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')
es_client.info()

ObjectApiResponse({'name': '583c4fc0bca3', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'LHX53ZB7QOG5C_ndt2NpGg', 'version': {'number': '8.17.6', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'dbcbbbd0bc4924cfeb28929dc05d82d662c527b7', 'build_date': '2025-04-30T14:07:12.231372970Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

In [6]:
index_name = 'course-questions'
try: 
    es_client.indices.create(index=index_name, body=index_settings)
except:
    print('some error')

In [7]:
from tqdm.auto import tqdm

In [8]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [9]:
es_client.cat.count(index=index_name, params={"format": "json"})[0]['count']

/tmp/ipykernel_67751/1044140496.py:1: DeprecationWarning: The 'params' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es_client.cat.count(index=index_name, params={"format": "json"})[0]['count']


'948'

In [10]:
def search(query):
    # index settings
    index_settings = {
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        },
        "mappings": {
            "properties": {
                "text": {"type": "text"},
                "section": {"type": "text"},
                "question": {"type": "text"},
                "course": {"type": "keyword"} 
            }
        }
    }
    # build search query
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs

In [11]:
print(datetime.now())
print(rag('how do i run kafka?')) # ollama version
print(datetime.now())

2025-06-24 23:03:17.910123
llama3.2:1b
Based on the CONTEXT, I can answer the QUESTION:

You need to install the 'dlt[duckdb]' package and ensure that duckdb is installed locally by running a command like !pip install dlt[duckdb] in your terminal. 

The answer is: To run the provided code, ensure that the 'dlt[duckdb]' package is installed and also have duckdb pip installed (even before loading the duckdb package).
2025-06-24 23:03:56.572429


In [12]:
try:
    genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
except KeyError:
    print('set the GOOGLE_API_KEY environment variable first')

In [13]:
def llm_google(prompt):
    model_name = 'gemma-3-1b-it'
    print(model_name)
    model = genai.GenerativeModel(model_name)
    response = model.generate_content(prompt)
    return response.text # google version

def rag_google(query):
    search_result = search(query)
    prompt = build_prompt(query, search_result)
    answer = llm_google(prompt)
    return answer

In [14]:
if False:
    print(datetime.now())
    print(rag_google('how do i run kafka?')) # google version
    print(datetime.now())

In [15]:
# Q1

!curl localhost:9200

{
  "name" : "583c4fc0bca3",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "LHX53ZB7QOG5C_ndt2NpGg",
  "version" : {
    "number" : "8.17.6",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "dbcbbbd0bc4924cfeb28929dc05d82d662c527b7",
    "build_date" : "2025-04-30T14:07:12.231372970Z",
    "build_snapshot" : false,
    "lucene_version" : "9.12.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [16]:
# Q2

print('index')

index


In [17]:
# Q3

query = 'How do execute a command on a Kubernetes pod?'
fields = ['question^4', 'text']
size = 20
search_query = {
    "size": size,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": fields,
                    "type": "best_fields"
                }
            },
        }
    }
}
response = es_client.search(index=index_name, body=search_query)

result_docs = []
for hit in response['hits']['hits']:
    result_docs.append(hit)

result_docs

[{'_index': 'course-questions',
  '_id': 'eE8upJcBwcE6Xp-kV_aY',
  '_score': 44.50556,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'B08upJcBwcE6Xp-kWve-',
  '_score': 35.433445,
  '_source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke',
   'section': '10. Kubernetes and TensorFlow Serving',
   'question': 'Kubernetes-dashboard',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'mE8upJcBwcE6Xp-kWPZd',
  '_score': 33.70974,
  '_source': {'text': 'You 

In [18]:
# Q4

query = 'How do copy a file to a Docker container?'
fields = ["question^3", 'text', 'section']
course = 'machine-learning-zoomcamp'
size = 3
search_query = {
    "size": size,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": fields,
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": course
                }
            }
        }
    }
}
response = es_client.search(index=index_name, body=search_query)

result_docs = []
for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

result_docs

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [19]:
# Q5

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
context = ''
for doc in result_docs:
    context = context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"
prompt = prompt_template.format(question=query, context=context).strip()
print(prompt)
print('\n')
print(len(prompt))

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do copy a file to a Docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/contain

In [20]:
# Q6

import tiktoken
encoding = tiktoken.encoding_for_model('gpt-4o')
len(encoding.encode(prompt))

320

In [21]:
# bonus Q7

print(datetime.now())
print(rag('how do i run kafka?')) # ollama version
print(datetime.now())

2025-06-24 23:05:54.238302
llama3.2:1b
Based on the CONTEXT, I'm unable to find an answer for how to run Kafka. The CONTEXT only discusses installing dependencies for running code and checking compatibility of local and container Spark versions.
2025-06-24 23:05:58.088566


In [22]:
# bonus Q8

def compute_cost(num_requests, num_inp_toks, num_out_toks, rate_per_1k_inp_toks, rate_per_1k_out_toks):
    inp_cost_per_request = num_inp_toks/1e3*rate_per_1k_inp_toks
    out_cost_per_request = num_out_toks/1e3*rate_per_1k_out_toks
    total_cost = num_requests*(inp_cost_per_request+out_cost_per_request)
    return total_cost

compute_cost(1e3, 150, 250, 0.005, 0.015)

4.5